In [81]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint

In [82]:
house_dict = []

for page in range(19, 21):
    linked = 'https://www.zillow.com/ca/townhomes/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-136.3134839375,"east":-102.2998120625,"south":27.484807219926367,"north":46.19194120230612},"mapZoom":5,"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"filterState":{"isComingSoon":{"value":false},"isAuction":{"value":false},"isPreMarketForeclosure":{"value":false},"isPreMarketPreForeclosure":{"value":false},"isMakeMeMove":{"value":false},"isSingleFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isApartment":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            # Identify and return house data
            listing_type = str(result.a.get('aria-label').split(",")[0])
            price = result.find('div', class_="list-card-price").text
            full_address = result.find('h3', class_="list-card-addr").text
            street = result.find('h3', class_="list-card-addr").text.split(",")[0]
            city = result.find('h3', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            try:
                state = result.find('h3', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('h3', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            if (state == "null"):
                full_address = loop_soup.find('div', class_="ds-price-change-address-row ds-collapse-row").find("h1", class_="ds-address-container").text.split("plan:")[1].strip().replace("\xa0", " ")
                city = loop_soup.find('div', class_="ds-price-change-address-row ds-collapse-row").find("h1", class_="ds-address-container").find_all("span")[1].text.split(",")[0].replace("\xa0", "")
                state = loop_soup.find('div', class_="ds-price-change-address-row ds-collapse-row").find("h1", class_="ds-address-container").find_all("span")[1].text.split(" ")[-2]
                zip_code = loop_soup.find('div', class_="ds-price-change-address-row ds-collapse-row").find("h1", class_="ds-address-container").find_all("span")[1].text.split(" ")[-1]
            try:
                house_type = loop_result1.select('li')[0].text.split(":")[1]
            except IndexError:
                house_type = 'null'
            try:
                year_build = loop_result1.select('li')[1].text.split(":")[1]
            except IndexError:
                year_build = 'null'
            try:
                lot = loop_result1.select('li')[5].text.split(":")[1]
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""))
                elif (str(lot.split(" ")[1]) == "acres"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', "")) * 43560
                else:
                    lot_conversion_to_sqft == "null"
            except IndexError:
                lot = 'null'
                lot_conversion_to_sqft = 'null'

            try:
                zillow_price_per_sqft = loop_result1.select('li')[6].text.split(":")[1]
            except IndexError:
                zillow_price_per_sqft = 'null'
            zestimate = loop_soup.find('span', class_="ds-estimate-value").text

            
            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "House_type": house_type,
                "Year_build": year_build,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "Zestimate": zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            
        except AttributeError as e:
            continue

print(house_dict)

[{'Listing_type': 'New construction', 'Price': '$508,190', 'Full_address': '7435 Solstice Pl, Rancho Cucamonga, CA', 'Street': '7435 Solstice Pl', 'City': 'RanchoCucamonga', 'State': 'CA', 'Zip_code': 'null', 'Bed': '3', 'Bath': '3', 'Square_Feet': '1,851', 'House_type': 'Townhouse', 'Year_build': '2019', 'Lot': 'No Data', 'Lot_conversion_to_sqft': 58806.00000000001, 'Zestimate': '$546,607', 'Zillow_price/sqft': '$275', 'House_website': 'https://www.zillow.com/homedetails/7435-Solstice-Pl-Rancho-Cucamonga-CA-91739/2082390896_zpid/'}, {'Listing_type': 'New construction', 'Price': '$749,000', 'Full_address': '1144 S Jacobs Dr, Mountain House, CA', 'Street': '1144 S Jacobs Dr', 'City': 'MountainHouse', 'State': 'CA', 'Zip_code': 'null', 'Bed': '3', 'Bath': '4', 'Square_Feet': '3,101', 'House_type': 'Townhouse', 'Year_build': '2019', 'Lot': '5,553 sqft', 'Lot_conversion_to_sqft': 5553.0, 'Zestimate': '$729,481', 'Zillow_price/sqft': '$242', 'House_website': 'https://www.zillow.com/homedeta

In [83]:
house_data = pd.DataFrame(house_dict)
house_data

,Bath,Bed,City,Full_address,House_type,House_website,Listing_type,Lot,Lot_conversion_to_sqft,Price,Square_Feet,State,Street,Year_build,Zestimate,Zillow_price/sqft,Zip_code
0,3,3,RanchoCucamonga,"7435 Solstice Pl, Rancho Cucamonga, CA",Townhouse,https://www.zillow.com/homedetails/7435-Solsti...,New construction,No Data,58806,"$508,190","1,851",CA,7435 Solstice Pl,2019,"$546,607",$275,null
1,4,3,MountainHouse,"1144 S Jacobs Dr, Mountain House, CA",Townhouse,https://www.zillow.com/homedetails/1144-S-Jaco...,New construction,"5,553 sqft",5553,"$749,000","3,101",CA,1144 S Jacobs Dr,2019,"$729,481",$242,null
2,3,3,Fremont,"40719 Penstemon Ter, Fremont, CA",null,https://www.zillow.com/community/boulevard-hei...,New construction,null,null,"$1,016,990+","1,975",CA,40719 Penstemon Ter,null,None,null,null
3,4,4,Dublin,"Residence Four, Boulevard Madison, Dublin, C...",Townhouse,https://www.zillow.com/community/boulevard-mad...,New construction,No Data,null,"$899,000+","2,128",CA,Residence Four Plan,2019,"$889,575",$422,94568
4,3,3,Carlsbad,"2815 Luciernaga St, Carlsbad, CA 92009",Townhouse,https://www.zillow.com/homedetails/2815-Lucier...,Townhouse for sale,"10,153 sqft",10153,"$649,900","1,266",CA,2815 Luciernaga St,1978,"$642,735",$513,92009
5,3,3,Whittier,"13228 Cullen St APT B, Whittier, CA 90602",Townhouse,https://www.zillow.com/homedetails/13228-Culle...,Townhouse for sale,0.78 acres,33976.8,"$515,000","1,559",CA,13228 Cullen St APT B,1987,"$515,003",$330,90602
6,2.5,2,Ontario,"Plan 2, Holiday at Emerald Park, Ontario, CA 9...",Townhouse,https://www.zillow.com/community/holiday-at-em...,New construction,No Data,33976.8,"$357,400+","1,048",CA,Plan 2 Plan,2019,"$357,400",$341,91761
7,3,3,Fremont,"37321 Hatchling Ter, Fremont, CA",3,https://www.zillow.com/community/renato/209443...,New construction,0,33976.8,"$928,990+","1,774",CA,37321 Hatchling Ter,3,None,"1,774 sqft",null
8,3,3,Oceanside,"Residence 3, Hideaway at St. Cloud, Oceanside,...",Townhouse,https://www.zillow.com/community/hideaway-at-s...,New construction,No Data,33976.8,"$569,990+","1,685",CA,Residence 3 Plan,2019,"$543,750",$338,92056
9,3,3,Oceanside,"Residence 2, Lucero at Pacific Ridge, Oceansid...",Townhouse,https://www.zillow.com/community/lucero-at-pac...,New construction,No Data,33976.8,"$496,990+","1,563",CA,Residence 2 Plan,2019,"$540,000",$318,92056


In [84]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Q4-2019_Townhouse_Data_page19-20.csv", index=False, header=True)